# Test RC Input Logging from second RX

In [1]:
from flightdata import Flight, Fields, CIDTypes

surfaces = {
    'r_aileron': 'tx_controls_1',
    'l_aileron': 'tx_controls_2',
    'elevator': 'tx_controls_3',
    'rudder': 'tx_controls_4',
}

deflection_bins = {key: Flight.from_log('../../data/logs/' + key + '.BIN', False) for key in surfaces.keys() }

In [2]:
from geometry import Point
from geometry.point import cross_product

import numpy as np
import pandas as pd
def get_angle(x1, y1, z1, x2, y2, z2):
    a = Point(x1, y1, z1)
    b = Point(x2, y2, z2)
    sinthetanhat = cross_product(a, b) / (abs(a) * abs(b) )
    return sinthetanhat.y

def make_deflection_df(bin, tx_control):
    angledata = bin.read_fields(Fields.ACCELERATION)
    
    angledata.columns = 'x2,y2,z2'.split(',')
    angledata.insert(0, 'z1', angledata.iloc[0].z2)
    angledata.insert(0, 'y1', angledata.iloc[0].y2)
    angledata.insert(0, 'x1', angledata.iloc[0].x2)

    angledata['angle'] = np.degrees(np.arcsin(np.vectorize(get_angle)(*tuple(angledata.transpose().to_numpy()))))


    angledata['input_pwm'] = bin.data[tx_control]
    return angledata[['angle', 'input_pwm']]

deflection_dfs = {surf: make_deflection_df(deflection_bins[surf], surfaces[surf]) for surf in surfaces.keys()}

deflection_dfs

<ipython-input-2-b59bf23677f6>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angledata['angle'] = np.degrees(np.arcsin(np.vectorize(get_angle)(*tuple(angledata.transpose().to_numpy()))))
<ipython-input-2-b59bf23677f6>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  angledata['input_pwm'] = bin.data[tx_control]


{'r_aileron':                angle  input_pwm
 time_index                     
 0.000000    0.000000     1568.0
 0.100116   -0.080161     1568.0
 0.200054   -0.065247     1569.0
 0.300021   -0.031688     1568.0
 0.400053   -0.075404     1568.0
 ...              ...        ...
 103.500533 -0.044822     1569.0
 103.600037  0.594226     1570.0
 103.700040 -0.502643     1570.0
 103.800022  1.504383     1569.0
 103.900014 -1.084316     1570.0
 
 [1040 rows x 2 columns],
 'l_aileron':                angle  input_pwm
 time_index                     
 0.000000    0.000000     1491.0
 0.100005   -0.190880     1491.0
 0.200025   -0.080024     1490.0
 0.300034   -0.213318     1491.0
 0.400039   -0.109133     1491.0
 ...              ...        ...
 98.299998  -1.592800     1490.0
 98.399993  -0.405394     1490.0
 98.499997   0.385615     1491.0
 98.599996  -0.605019     1491.0
 98.699876   1.113551     1490.0
 
 [988 rows x 2 columns],
 'elevator':                angle  input_pwm
 time_index     

In [3]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

angledata = deflection_dfs['rudder']
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(x=angledata.index, y=angledata.input_pwm, name="pwm"),secondary_y=False)
fig.add_trace(go.Scatter(x=angledata.index, y=angledata.angle, name="angle"),secondary_y=True)

fig.update_xaxes(title_text="xaxis title")

fig.update_yaxes(title_text="input_pwm", secondary_y=False)
fig.update_yaxes(title_text="angle(deg)", secondary_y=True)

fig.show()

In [4]:
rate_regions = {
    'rudder':{
        'landing': [0,38],
        'line': [38,66],
        'radius1':[38,66],
        'radius2':[66,78],
        'radius3':[78,88],
        'stallturn': [108,126],
        'spin': [88,108],
        'snap': [126,140]
    },
    'elevator':{
        'landing': [0, 10.5],
        'line': [26, 36],
        'radius1': [37, 44],
        'radius2': [44, 48],
        'radius3': [48, 54],
        'stallturn': [48, 54],
        'spin': [0, 10.5],
        'snap': [44, 48]
    },
    'r_aileron':{
        'landing': [12, 33],
        'line': [33,51],
        'radius1':[51,60],
        'radius2':[60,68],
        'radius3':[68,75],
        'stallturn': [75, 83],
        'spin': [83, 90],
        'snap': [92, 98]
    },
    'l_aileron':{
        'landing': [0, 15],
        'line': [15,28],
        'radius1':[28,37],
        'radius2':[37,44],
        'radius3':[44,55],
        'stallturn': [55,67],
        'spin': [67,78],
        'snap': [67,92]
    }
    }



In [5]:


class RateInfo():
    def __init__(self, angledata, regions):
        self.centre = angledata.input_pwm.median()
        self.func = np.polyfit(angledata.input_pwm, angledata.angle,3)
        self.ratedata = {key: angledata.loc[value[0]:value[1]] for key, value in regions.items()}
        self.rates = {key: [value.input_pwm.min(), value.input_pwm.max()] for key, value in self.ratedata.items()}
        self.angles = {key: [self.deflection(value[0]), self.deflection(value[1])] for key, value in self.rates.items()}        

    def deflection(self, pwm):
        return self.func[0] * pwm**3 + self.func[1] * pwm**2 + self.func[2] * pwm + self.func[3]

rateinfo = {}
for key in rate_regions.keys():
    rateinfo[key] = RateInfo(deflection_dfs[key], rate_regions[key])


In [6]:

for key, value in rateinfo.items():
    fig = go.Figure()

    pwm_values = np.linspace(value.rates['landing'][0], value.rates['landing'][1]) 
    deflections = np.vectorize(value.deflection)(pwm_values)


    line = fig.add_trace(go.Scatter(x=pwm_values, y=deflections, name='full range'))

    for rate in value.rates.keys():
        if rate in 'landing,line,radius1'.split(','):
            fig.add_trace(go.Scatter(
                x=value.rates[rate], 
                y=value.angles[rate], 
                text=["{:.2f}".format(val) for val in value.angles[rate]], 
                mode='markers+text',
                textposition="bottom right", 
                name=rate
                ))

    fig.update_layout(title= key + ' Deflection vs pwm value')
    fig.update_xaxes(title_text="signal pulse width")

    fig.update_yaxes(title_text="deflection (degrees)")
    fig.show()